In [1]:
import torch
from torch import nn
import logging
from credit.models import load_model
import yaml
import os
import xarray as xr
from os.path import join
# from credit.models.skebs_model import SKEBS
from matplotlib import pyplot as plt

logger = logging.getLogger(__name__)

In [ ]:
model_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_fcnn_scaled_train"
orig_ckpt = torch.load(join(model_dir, "checkpoint.pt"), map_location="cpu")
state_dict = orig_ckpt['model_state_dict']

In [ ]:
# save_loc = "/glade/work/dkimpara/CREDIT_runs/skebs_1dg_cesm/"
# config = "/glade/work/dkimpara/CREDIT_runs/skebs_1dg_cesm_frozen/model.yml"
config = "/glade/work/dkimpara/CREDIT_runs/skebs_1dg_cesm_tuning/model.yml"
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)
save_loc = os.path.expandvars(conf["save_loc"])

In [ ]:
ckpt = torch.load(join(save_loc, "checkpoint.pt"),  map_location=torch.device('cpu'))
state_dict = ckpt["model_state_dict"]

In [ ]:
postblock_keys = [key for key in state_dict.keys() if 'postblock' in key]
print(postblock_keys)

In [ ]:
spectral_filter.shape

In [ ]:
berner_values = [0.125, 0.083, -1.27, 10e-4, 0.1]
for i, key in enumerate(postblock_keys[:-2]):
    print(f'{key}: {state_dict[key]:.3f} | Berner value: {berner_values[i]}')

min_backscatter, max_backscatter = state_dict[postblock_keys[-1]].min(), state_dict[postblock_keys[-1]].max()
print(f"min backscatter: {min_backscatter:.3f}, max backscatter {max_backscatter:.3f}")

spectral_filter = state_dict[postblock_keys[-2]].squeeze()
plt.plot(range(len(spectral_filter)), spectral_filter)
plt.title("spectral filter coefficients")

In [ ]:
num_keys = len(state_dict.keys()) - len(postblock_keys)

base_model_keys = [key for key in state_dict.keys() if 'postblock' not in key]

num_close = 0
for key in base_model_keys:
    num_close += 1 if torch.allclose(o_state_dict[key], state_dict[key], atol=1e-2) else 0
print(num_close/num_keys)

In [ ]:
for key in postblock_keys:
    print(key, state_dict[key])



In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

def remove_nan_rows(df):
    """
    Removes rows from the DataFrame that contain any NaN values.
    
    Parameters:
    df (pandas.DataFrame): The input DataFrame.

    Returns:
    pandas.DataFrame: The DataFrame with rows containing NaN values removed.
    """
    return df.dropna()

train_log2 = pd.read_csv(join(save_loc, 'training_log.csv'))
train_log2 = remove_nan_rows(train_log2)

plt.figure()

for col in ['train_loss', 'valid_loss']:
    data = train_log2[col]
    plt.plot(data, label=col)

plt.legend()

plt.figure()

for col in ['train_acc', 'valid_acc']:
    data = train_log2[col]
    plt.plot(data, label=col)

plt.legend()